In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras 
from tensorflow.keras import layers


In [ ]:
! pip install numpy 

In [ ]:
! pip install pandas 

In [ ]:
! pip install matplotlib

In [ ]:
! pip install tensorflow

In [ ]:
data_train_path = r'C:\Users\ABHINAV KUMAR\Desktop\Image Classification\Fruits_Vegetables\train'
data_test_path = r'C:\Users\ABHINAV KUMAR\Desktop\Image Classification\Fruits_Vegetables\test'
data_val_path = r'C:\Users\ABHINAV KUMAR\Desktop\Image Classification\Fruits_Vegetables\validation'

In [ ]:
img_width=180
img_height=180

In [ ]:
data_train=tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=False)

In [ ]:
data_cat=data_train.class_names

In [ ]:
data_val =tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    image_size=(img_width, img_height), 
    batch_size=32,
    validation_split=False,
    shuffle=False)

In [ ]:
data_test = tf.keras.utils.image_dataset_from_directory(
    data_test_path,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=False,
    shuffle=False)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in data_train.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(data_cat[labels[i]])
        plt.axis("off")

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_width, img_height, 3)),
    
    # First Conv Block
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    # Second Conv Block
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    # Third Conv Block
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    # Fourth Conv Block
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    # Dense layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(len(data_cat), activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
epochs = 15

# Add callbacks for better training
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-7
)

history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=epochs,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
model.summary()

In [ ]:
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
image = 'Banana.jpg'
image = keras.utils.load_img(
    image, target_size=(img_width, img_height)
)
img_arr = tf.keras.utils.img_to_array(image)
img_bat = tf.expand_dims(img_arr, 0)  # Create a batch

In [ ]:
predict = model.predict(img_bat)
predict

In [ ]:
score = tf.nn.softmax(predict)

In [ ]:
print('Veg/Fruit in image is {} with accuracy of {:.2f}%'.format(data_cat[np.argmax(score)], np.max(score)*100))

In [ ]:
model.save('Image_classify.keras')